In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Jamestown,42.0970,-79.2353,75.06,46,75,1.01,US,broken clouds
1,1,Buraydah,26.3260,43.9750,96.17,9,0,8.25,SA,clear sky
2,2,Ushuaia,-54.8000,-68.3000,39.42,80,99,5.95,AR,overcast clouds
3,3,Puerto Ayora,-0.7393,-90.3518,69.55,90,100,11.01,EC,overcast clouds
4,4,Mikuni,36.2171,136.1519,73.20,94,99,7.70,JP,overcast clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))


max_temp = float(input("What is the maximum temperature you would like for your trip? "))




What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
 #3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Jamestown,42.0970,-79.2353,75.06,46,75,1.01,US,broken clouds
8,8,Butaritari,3.0707,172.7902,81.63,67,72,14.92,KI,broken clouds
14,14,Kieta,-6.2167,155.6333,77.63,85,100,4.38,PG,overcast clouds
16,16,Albany,42.6001,-73.9662,76.59,68,19,5.14,US,few clouds
19,19,Kavieng,-2.5744,150.7967,81.70,73,83,14.97,PG,broken clouds
20,20,Jalu,29.0331,21.5482,83.46,27,0,8.08,LY,clear sky
21,21,Gizo,-8.1030,156.8419,81.23,77,44,6.82,SB,scattered clouds
25,25,Ormara,25.2088,64.6357,80.67,83,0,10.98,PK,clear sky
26,26,Marang,5.2056,103.2059,78.69,82,93,2.57,MY,overcast clouds
30,30,Merauke,-8.4667,140.3333,76.32,93,81,10.33,ID,broken clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                254
City                   254
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Country                251
Current Description    254
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                3
Current Description    0
dtype: int64

In [24]:
# create a new DataFrame that doesn’t have empty rows
clean_df =preferred_cities_df

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,75.06,broken clouds,42.0970,-79.2353,
8,Butaritari,KI,81.63,broken clouds,3.0707,172.7902,
14,Kieta,PG,77.63,overcast clouds,-6.2167,155.6333,
16,Albany,US,76.59,few clouds,42.6001,-73.9662,
19,Kavieng,PG,81.70,broken clouds,-2.5744,150.7967,
20,Jalu,LY,83.46,clear sky,29.0331,21.5482,
21,Gizo,SB,81.23,scattered clouds,-8.1030,156.8419,
25,Ormara,PK,80.67,clear sky,25.2088,64.6357,
26,Marang,MY,78.69,overcast clouds,5.2056,103.2059,
30,Merauke,ID,76.32,broken clouds,-8.4667,140.3333,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [57]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,75.06,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
8,Butaritari,KI,81.63,broken clouds,3.0707,172.7902,Isles Sunset Lodge
14,Kieta,PG,77.63,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
16,Albany,US,76.59,few clouds,42.6001,-73.9662,
19,Kavieng,PG,81.70,broken clouds,-2.5744,150.7967,Nusa Island Retreat
...,...,...,...,...,...,...,...
693,Rosamorada,MX,87.62,overcast clouds,22.1222,-105.2058,Hotel Gonzalez
694,Bosaso,SO,89.89,clear sky,11.2842,49.1816,Red Sea Hotel
695,Columbus,US,80.87,scattered clouds,39.9612,-82.9988,DoubleTree Suites by Hilton Hotel Columbus Dow...
696,Umm Lajj,SA,85.57,clear sky,25.0213,37.2685,Moon light Furnished Units


In [72]:
 #7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df[hotel_df["Hotel Name"]!= '']

In [74]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,75.06,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
8,Butaritari,KI,81.63,broken clouds,3.0707,172.7902,Isles Sunset Lodge
14,Kieta,PG,77.63,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
19,Kavieng,PG,81.70,broken clouds,-2.5744,150.7967,Nusa Island Retreat
20,Jalu,LY,83.46,clear sky,29.0331,21.5482,صاحبة الفخامة
...,...,...,...,...,...,...,...
693,Rosamorada,MX,87.62,overcast clouds,22.1222,-105.2058,Hotel Gonzalez
694,Bosaso,SO,89.89,clear sky,11.2842,49.1816,Red Sea Hotel
695,Columbus,US,80.87,scattered clouds,39.9612,-82.9988,DoubleTree Suites by Hilton Hotel Columbus Dow...
696,Umm Lajj,SA,85.57,clear sky,25.0213,37.2685,Moon light Furnished Units


In [75]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [76]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F </dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [77]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))